In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from funkagent import agents
from dotenv import load_dotenv
import asyncio

In [3]:
# Load config file
load_dotenv()

True

In [4]:
# Define simple agent

In [5]:
# create a simple function for testing with
def circumference_calculator(radius: float, something: float = 4.4) -> float:
    """Calculates the circumference of a circle given the radius.

    :param radius: The radius of the circle as a single, positive number (float) - no math calculations (*,+,-,/) as input allowed.
    :return: The circumference of the circle
    """
    return 2 * 3.14 * radius

In [18]:
my_agent = agents.Agent(
    functions=[circumference_calculator]
)

In [23]:
await my_agent.ask("What is the circumference of a circle with radius 5 in one sentence?")

..

'The circumference of a circle with a radius of 5 is approximately 31.4 units.'

In [22]:
my_agents = [agents.Agent(functions=[circumference_calculator]) for _ in range(3)]

questions = [
    "What is love in one sentence?",
    "What is the circumference of a circle with radius 7 in one sentence?",
    "Who is Messi in one sentence?",
    "What is the circumference of a circle with radius 9 in one sentence?",
]

results = await asyncio.gather(*[agent.ask(question) for question, agent in zip(questions, my_agents)])

for answer, question in zip(results, questions):
    print("\n")
    print(question)
    print("->")
    print(answer)

....What is love in one sentence?
->
Love is a complex and multifaceted emotion that encompasses deep affection, care, and connection between individuals, often characterized by selflessness, understanding, and a willingness to support and nurture one another.


What is the circumference of a circle with radius 7 in one sentence?
->
The circumference of a circle with a radius of 7 is approximately 43.96 units.


Who is Messi in one sentence?
->
Messi is an Argentine professional soccer player widely regarded as one of the greatest of all time, known for his incredible skills, record-breaking achievements, and his long and successful career at FC Barcelona.




In [22]:
# Add cognitive search as vector store